#Importando Bibliotecas e Carregando o Dataset


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file_path = '/content/CONSUMO_2024.csv'
df = pd.read_csv(file_path)

df.head()


#2. Exploração e análise do dataset


In [ ]:
df.isnull().sum()

df.fillna(df.mean(), inplace=True)

label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Alvo

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print(f"Número de amostras: {df.shape[0]}")
print(f"Número de features: {df.shape[1] - 1}")
print(f"Número de classes na variável alvo: {len(set(y))}")


# 3. Desenvolvimento do modelo


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)))


#4. Compilação do modelo


In [ ]:
from tensorflow.keras.metrics import Accuracy, AUC

def f1_score_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    precision = tp / (tf.reduce_sum(tf.cast(y_pred, 'float'), axis=0) + tf.keras.backend.epsilon())
    recall = tp / (tf.reduce_sum(tf.cast(y_true, 'float'), axis=0) + tf.keras.backend.epsilon())
    f1_score = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return tf.reduce_mean(f1_score)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', AUC(name='auc'), f1_score_metric])


#5. Treinamento do modelo


In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2, verbose=2)


# 6. Avaliação do Modelo


In [ ]:
evaluation = model.evaluate(X_test, y_test, verbose=2)
print(f"Acurácia no conjunto de teste: {evaluation[1]:.4f}")
print(f"AUC no conjunto de teste: {evaluation[2]:.4f}")
print(f"F1 Score no conjunto de teste: {evaluation[3]:.4f}")
